## Ecuación de Ondas utilizando diferencias finitas y el método de líneas

Autores: Oscar Reula, Joaquín Pelle, Pablo Montes

Utilizaremos el paquete `DifferentialEquations.jl`. 
Este notebook contiene modificaciones de ejemplos de las siguientes dos páginas:

https://tutorials.sciml.ai/html/introduction/03-optimizing_diffeq_code.html

http://juliamatrices.github.io/BandedMatrices.jl/latest/#Creating-banded-matrices-1

## Método de líneas
Resolveremos ecuaciones hiperbólicas utilizando el *método de líneas* y aproximaciones de *diferencias finitas*. Esto significa que si partimos de un sistema escalar de la forma

\begin{equation}
u_t = u_x
\end{equation}

y lo aproximaremos en un comienzo por un operador de diferencias finitas $D_x$ en la dirección espacial, para obtener

\begin{equation}
v_t = D_x\; v.
\end{equation}

Donde $v$ es una versión de $u$ discretizada *solo* en el espacio. Esto es, si tomamos una grilla de $N$ puntos, $v$ será un vector de $N$ elementos que depende contínuamente en el tiempo. 

En este punto, tenemos efectivamente un sistema de ecuaciones ordinarias de dimensión $N$, al cual podemos aproximar utilizando un integrador ODE adecuado. De esta manera terminamos con una discretización en espacio y en tiempo.

## Ecuación de Onda

Resolveremos la ecuación de onda 
\begin{equation}
\phi_{tt} = \phi_{xx}
\end{equation}

Para expresarla en forma estándar definiremos dos nuevas variables, $u := \phi_x$ y $v := \phi_t$. Este sistema entonces se convierte en

\begin{eqnarray}
\phi_t & = & v \\
v_t & = & u_x \\
u_t & = & v_x,
\end{eqnarray}

donde hemos usado que $u_t := \phi_{xt} = \phi_{tx} := v_x$, y $v_t :=\phi_{tt} = \phi_{xx} = u_x$.  

Dado que la ecuación para $\phi$ puede ser integrada una vez conocidos $(u,v)$, y que $\phi$ no es necesario para resolver el resto del sistema, lo ignoraremos por el momento.

### Diagonalización del sistema

Si definimos las variables $u^{+} = u+v$ y $u^{-} = u-v$ podemos obtener un sistema diagonalizado, 

\begin{eqnarray}
u^+_t & = & u^+_x \\
u^-_t & = & -u^-_x,
\end{eqnarray}

donde la solución son dos ondas independientes, $u^{+}$ hacia la izquierda y $u^{-}$ hacia la derecha:

\begin{eqnarray}
u^{+}(x,t) = u^{+}_0(x+t)\\
u^{-}(x,t) = u^{-}_0(x-t)
\end{eqnarray}

Por lo tanto, las soluciones del sistema original serán combinaciones lineales de estas dos soluciones, las cuales dependen del dato inicial.

Por ejemplo, si tomamos como dato inicial $\phi_0(x) = e^{-x^{2}}$ y $\phi_t(x, t=0) = 0$ con condiciones periódicas de contorno en el intervalo $(-4,4)$, la solución exacta será

In [ ]:
using Plots
N = 201
dx = 8.0/(N-1)
x = range(-4.0, 4.0, length = N) #valores espaciales
tiempos = range(0, 8, length = 200)              #valores temporales

function return_to_interval(xini::Number, xfin::Number, x)
    #Esta función toma un valor x y lo retorna al intervalo [xini, xfin)
    length = xfin - xini
    offset = x + xini
    factor = offset / length
    x = xini + (factor - floor(factor))*length
end

function up_0(x)
    #(u^+)_0
    x = return_to_interval(-4,4,x)
    return -2*x*exp(-(x)^2) 
end
function um_0(x)
    #(u^-)_0
    x = return_to_interval(-4,4,x)
    return -2*x*exp(-(x)^2)
end

anim = @animate for t in tiempos
    up = up_0.(x.+t)
    um = um_0.(x.-t)
    u = 0.5*(up+um)
    v = 0.5*(up-um)
    ϕ = [dx*sum(u[1:i]) for i in 1:N]
    ϕ = ϕ .- minimum(ϕ)
    lfs = 10
    p1 = plot(x, ϕ, label = "\$ \\phi \$", ylim = (-0.1, 1.1), xlim = (-4,4), 
        xlabel = "\$x\$",legendfontsize = lfs)
    title!(p1, "Sistema original")
    p2 = plot(x, u, label = "\$u\$",legendfontsize = lfs)
    plot!(p2, x, v, label = "\$v\$", ylim = (-1,1), xlim = (-4,4),
        xlabel = "\$x\$", legendfontsize = lfs)
    title!(p2, "Nuevo Sistema")
    p3 = plot(x, up, label = "\$u^{+}\$", legendfontsize = lfs)
    plot!(p3, x, um, label = "\$u^{-}\$", ylim = (-1,1), xlim = (-4,4),
        xlabel = "\$x\$", legendfontsize = lfs)
    title!(p3, "Nuevo Sistema Diagonalizado")
    plot(p1, p2, p3, layout = (3,1), size=(400,600))
end
gif(anim, "Ejemplo_Onda.gif", fps = 30)

Estamos usando Julia, por lo que cargaremos algunos paquetes para manejar matrices, resolver ODEs y graficar.

In [ ]:
using OrdinaryDiffEq  #Esta es solo una parte del paquete DifferentialEquations
#using DifferentialEquations
using Plots
using LinearAlgebra
using BandedMatrices
using SparseArrays

Ahora agregamos algunos parámetros para la simulación. Algunos valores son abritrarios, y puede probar jugar con ellos. $N$ es el número de puntos en la discretización espacial. Resolveremos el problema *periódico*, por lo que si la grilla comienza en $1$ y termina en $N$, identificaremos los puntos $(N+1, N+2,...)$ con $(1, 2, ...)$ y los puntos $(0, -1, ...)$ con $(N, N-1, ...)$.

In [ ]:
N = 500            # Numeros de puntos en la discretización espacial
L = 1.0            # Intervalo espacial
dx = L/N           # dx
T = 10.            # Tiempo final
dt = 1. *dx        # Tomamos dt ≈ dx/speed_max, para mantener la condición CFL y
                   # Garantizar la estabilidad del algoritmo.

r0=zeros(N,2)      # Discretización de los campos u y v
x = [dx*i for i in 0:N-1];      # Coordenadas x, necesarias para determinar el dato inicial.

Ahora definimos los esquemas de diferencias finitas. Están implementados como matrices que multiplican los vectores de la solución. Las matrices están definidas como dispersas para mayor eficiencia computacional.
En general no es eficiente definir las derivadas numéricas con matrices, pero los casos con los que trabajaremos son bastante sencillos e ilustrativos.

In [ ]:
function create_D_2_per(N)
    D_2_per = sparse(Tridiagonal([-0.5 for i in 1:N-1],[0.0 for i in 1:N],[0.5 for i in 1:N-1]))
    D_2_per[1,end] = -0.5
    D_2_per[end,1] = 0.5
    dropzeros!(D_2_per)
    return D_2_per
end


function create_D2_2_per(N)
    D2_2_per = BandedMatrix{Float64}(Zeros(N,N), (N-1,N-1))
    D2_2_per[band(0)] .= -2.0
    D2_2_per[band(1)] .= 1.0
    D2_2_per[band(-1)] .= 1.0
    
    D2_2_per[band(N-1)] .= 1.0
    D2_2_per[band(-N+1)] .= 1.0
    
    D2_2_per = sparse(D2_2_per)
    dropzeros!(D2_2_per)
    return D2_2_per
end

function create_D_4_per(N)
    D_4_per = BandedMatrix{Float64}(Zeros(N,N), (N-1,N-1))
    D_4_per[band(0)] .= 0.0
    D_4_per[band(1)] .= 2.0/3.0
    D_4_per[band(-1)] .= -2.0/3.0
    D_4_per[band(2)] .= -1.0/12.0
    D_4_per[band(-2)] .= 1.0/12.0
    
    D_4_per[band(N-1)] .= -2.0/3.0
    D_4_per[band(N-2)] .= 1.0/12.0
    
    D_4_per[band(-N+1)] .= 2.0/3.0
    D_4_per[band(-N+2)] .= -1.0/12.0
    
    D_4_per = sparse(D_4_per)
    dropzeros!(D_4_per)
    return D_4_per
end

function create_D_6_per(N)
    D_6_per = BandedMatrix{Float64}(Zeros(N,N), (N-1,N-1))
    D_6_per[band(0)] .= 0.0
    D_6_per[band(1)] .= 3.0/4.0
    D_6_per[band(-1)] .= -3.0/4.0
    D_6_per[band(2)] .= -3.0/20.0
    D_6_per[band(-2)] .= 3.0/20.0
    D_6_per[band(3)] .= 1.0/60.0
    D_6_per[band(-3)] .= -1.0/60.0
    
    D_6_per[band(N-1)] .= -3.0/4.0
    D_6_per[band(N-2)] .= 3.0/20.0
    D_6_per[band(N-3)] .= -1.0/60.0
    
    D_6_per[band(-N+1)] .= 3.0/4.0
    D_6_per[band(-N+2)] .= -3.0/20.0
    D_6_per[band(-N+3)] .= 1.0/60.0
    
    D_6_per = sparse(D_6_per)
    dropzeros!(D_6_per)
    return D_6_per
end

function create_D_8_per(N)
    D_8_per = BandedMatrix{Float64}(Zeros(N,N), (N-1,N-1))
    D_8_per[band(0)] .= 0.0
    D_8_per[band(1)] .= +672.0/840.0
    D_8_per[band(-1)] .= -672.0/840.0
    D_8_per[band(2)] .= -168.0/840.0
    D_8_per[band(-2)] .= +168.0/840.0
    D_8_per[band(3)] .= +32.0/840.0
    D_8_per[band(-3)] .= -32.0/840.0
    D_8_per[band(4)] .= -3.0/840.0
    D_8_per[band(-4)] .= +3.0/840.0
    
    D_8_per[band(N-1)] .= -672.0/840.0
    D_8_per[band(N-2)] .= 168.0/840.0
    D_8_per[band(N-3)] .= -32.0/840.0
    D_8_per[band(N-4)] .= 3.0/840.0
    
    D_8_per[band(-N+1)] .= 672.0/840.0
    D_8_per[band(-N+2)] .= -168.0/840.0
    D_8_per[band(-N+3)] .= 32.0/840.0
    D_8_per[band(-N+4)] .= -3.0/840.0
    
    D_8_per = sparse(D_8_per)
    dropzeros!(D_8_per)
    return D_8_per
end

In [ ]:
println("Aproximación de segundo orden periódica de la derivada primera:")
println(create_D_2_per(8))
println("Aproximación de cuarto orden periódica de la derivada primera:")
println(round.(create_D_4_per(8), digits = 4))
println("Aproximación de sexto orden periódica de la derivada primera:")
println(round.(create_D_6_per(8), digits = 4))
println("Aproximación de octavo orden periódica de la derivada primera:")
println(round.(create_D_8_per(8), digits = 4))

Definimos ahora el lado derecho de las ecuaciones en el método de líneas, es decir, la discretización espacial.

In [ ]:
function F!(dr,r,p,t)
    # second order version
    dx, D = p
    h = 1. /dx
    u = @view r[:,1]
    v = @view r[:,2]
    du = @view dr[:,1]
    dv = @view dr[:,2]
    mul!(du, D, v, h, 0)  #du/dt = h*Dv
    mul!(dv, D, u, h, 0)  #dv/dt = h*Du
    #Nota: mul!(C, A, B, α, β) hace la operación α*A*B + β*C y la guarda en C
end

Ahora damos el dato inicial. En este caso particular, daremos uno en el que $u^{+}$ se anula. Por lo tanto, la única onda que debieramos ver es $u^{-}$ yendo hacia la derecha. Puede jugar y cambiar el dato inicial por otro.

**1)**
Juegue con el código adjunto (Wave_equation): a) Cambie condiciones iniciales; b) evoluciones a distintos tiempos; c) grafique distintas variables o funciones de las mismas; d) intente graficar una animación de la solución; e) pruebe distintas resoluciones espaciales y temporales (que sucede si toma dt muy grande?) Explique. Calcule la energía de la solución y vea qué tan bien se conserva.

**2)** Cree operadores differencia finita de orden 6 y 8. Aplíquelos a la ecuación de onda. Juntos con los ya presentes (orden 2 y 4), evolucione las ecuaciones hasta T=100 y compare los resultados con la solución exacta
(al cabo de períodos enteros la solución exacta coincide con el dato inicial).
Haga un plot de las distintas aproximaciones. 






In [ ]:
x0 = 0.4; x1 = 0.6
u = @view r0[:,1]
v = @view r0[:,2]
r02 = copy(r0)
u2 = @view r02[:,1]
v2 = @view r02[:,2]
#el at view lo que hace es renombrar una columna para que pueda ser llamada por otro nombre, pero no crea ninguna variable nueva
for i in 1:N
    x[i] = dx*(i-1)
    if x[i] > x0 && x[i] < x1
        u[i] = 1/(((x[i]-0.5)*75.0)^2+1.0)
        #lorenziana
        #u[i] = (x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250
        #polinomio
        #u[i] = (x[i]-x0)*5.0
        #recta
        #u[i] = exp(-(x[i]-0.5)^2/0.002)
        #gaussiana
        #u[i] = log(1.0/(1.0-(x[i]-x0)*5))-(x[i]-x0)*5*log((x[i]-x0)*5/(1.0-(x[i]-x0)*5))
        #logs
        v[i] = -u[i]
    end
end

for i in 1:N
    x[i] = dx*(i-1)
    if x[i] > x0 && x[i] < x1
        #u2[i] = 1/(((x[i]-0.5)*75.0)^2+1.0)
        #lorenziana
        #u[i] = (x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250
        #polinomio
        u2[i] = (x[i]-x0)*5.0
        #recta
        #u[i] = exp(-(x[i]-0.5)^2/0.002)
        #gaussiana
        #u[i] = log(1.0/(1.0-(x[i]-x0)*5))-(x[i]-x0)*5*log((x[i]-x0)*5/(1.0-(x[i]-x0)*5))
        #logs
        v2[i] = -u2[i]
    end
end
initial1 = plot()
plot!(initial1, x, r0, label = ["\$u\$" "\$v\$"], xlabel = "\$x\$", legendfontsize = 15,title="Lorenziana")
initial2 = plot()
plot!(initial2, x, r02, label = ["\$u\$" "\$v\$"], xlabel = "\$x\$", legendfontsize = 15, title="Recta")
plot(initial1,initial2,size=(1200,600))

Definimos dos porblemas con distinta precisión

In [ ]:
D_2_per = create_D_2_per(N)
D_4_per = create_D_4_per(N)
D_6_per = create_D_6_per(N)
D_8_per = create_D_8_per(N)

p2 = (dx, D_2_per)
p4 = (dx, D_4_per)
p6 = (dx, D_6_per)
p8 = (dx, D_8_per)
@time prob2 = ODEProblem(F!,r0,(0.0,T),p2);
@time prob4 = ODEProblem(F!,r0,(0.0,T),p4);
@time prob6 = ODEProblem(F!,r0,(0.0,T),p6);
@time prob8 = ODEProblem(F!,r0,(0.0,T),p8);

@time tprob2 = ODEProblem(F!,r02,(0.0,T),p2);
@time tprob4 = ODEProblem(F!,r02,(0.0,T),p4);
@time tprob6 = ODEProblem(F!,r02,(0.0,T),p6);
@time tprob8 = ODEProblem(F!,r02,(0.0,T),p8);

Ahora resolvemos:

In [ ]:
@time sol2 = solve(prob2,RK4(),dt=dt);
@time sol4 = solve(prob4,RK4(),dt=dt);

In [ ]:
@time sol6 = solve(prob6,RK4(),dt=dt);
@time sol8 = solve(prob8,RK4(),dt=dt);

In [ ]:
@time tsol2 = solve(tprob2,RK4(),dt=dt);
@time tsol4 = solve(tprob4,RK4(),dt=dt);

In [ ]:
@time tsol6 = solve(tprob6,RK4(),dt=dt);
@time tsol8 = solve(tprob8,RK4(),dt=dt);

Finalmente, graficamos la solución a distintos tiempos. Notemos que, como la velocidad de propagación es $1$ y el sistema es periódico y de longitud espacial $1$, esperamos que para $t = i*1.0$ recuperemos el dato inicial.

In [ ]:
secnd = plot()
for i in 0:4
    plot!(secnd,x, sol2(T*0.1*i)[:,1], label = "t = $(T*0.1*i)")
end
xlabel!("\$x\$")
ylabel!("\$u\$")

tsecnd = plot()
for i in 0:4
    plot!(tsecnd,x, tsol2(T*0.1*i)[:,1], label = "t = $(T*0.1*i)")
end
xlabel!("\$x\$")
ylabel!("\$u\$")

plot(secnd,tsecnd,title="Derivada de segundo orden",size=(1200,600))

In [ ]:
forth = plot()
for i in 0:4
    plot!(forth,x, sol4(T*0.1*i)[:,1], label = "t = $(T*0.1*i)")
end
xlabel!("\$x\$")
ylabel!("\$u\$")

tforth = plot()
for i in 0:4
    plot!(tforth,x, tsol4(T*0.1*i)[:,1], label = "t = $(T*0.1*i)")
end
xlabel!("\$x\$")
ylabel!("\$u\$")

plot(forth,tforth,title="Derivada de cuarto orden",size=(1200,600))

In [ ]:
sixth = plot()
for i in 0:4
    plot!(sixth,x, sol6(T*0.1*i)[:,1], label = "t = $(T*0.1*i)")
end
xlabel!("\$x\$")
ylabel!("\$u\$")

tsixth = plot()
for i in 0:4
    plot!(tsixth,x, tsol6(T*0.1*i)[:,1], label = "t = $(T*0.1*i)")
end
xlabel!("\$x\$")
ylabel!("\$u\$")

plot(sixth,tsixth,title="Derivada de sexto orden",size=(1200,600))

In [ ]:
oct = plot()
for i in 0:4
    plot!(oct,x, sol8(T*0.1*i)[:,1], label = "t = $(T*0.1*i)")
end
xlabel!("\$x\$")
ylabel!("\$u\$")

toct = plot()
for i in 0:4
    plot!(toct,x, tsol8(T*0.1*i)[:,1], label = "t = $(T*0.1*i)")
end
xlabel!("\$x\$")
ylabel!("\$u\$")

plot(oct,toct,title="Derivada de octavo orden",size=(1200,600))

## Una cosa que se puede ver, es que cuando la condición inicial tiene una discontinuidad, las aproximaciones empeoran bastante, en particular hay mucho ruido en donde la onda debería valer cero

## Animaciones

In [ ]:
Plt=copy(sol2(0.002)[:,1])
TT=floor(Int, T)
anim = @animate for j in 1:25:N*TT
    for i in 1:N-1
        if (i-j%N)>=0
            Plt[N-i]=sol2(j*dt)[N+j%N-i,1]
        else
            Plt[N-i]=sol2(j*dt)[-i+j%N,1]
        end
    end
    plot(Plt,ylim = (-0.5, 1.5),title="Solución Lorenziana orden 2",label = "t = $(j/N)")
end
gif(anim, "wave_deformation.gif", fps = 25)


In [ ]:
Plt=copy(sol6(0.002)[:,1])
TT=floor(Int, T)
anim = @animate for j in 1:25:N*TT
    for i in 1:N-1
        if (i-j%N)>=0
            Plt[N-i]=sol6(j*dt)[N+j%N-i,1]
        else
            Plt[N-i]=sol6(j*dt)[-i+j%N,1]
        end
    end
    plot(Plt,ylim = (-0.5, 1.5),title="Solución Lorenziana orden 6",label = "t = $(j/N)")
end
gif(anim, "wave_deformation2.gif", fps = 25)

In [ ]:
Plt=copy(tsol2(0.002)[:,1])
TT=floor(Int, T)
anim = @animate for j in 1:25:N*TT
    for i in 1:N-1
        if (i-j%N)>=0
            Plt[N-i]=tsol2(j*dt)[N+j%N-i,1]
        else
            Plt[N-i]=tsol2(j*dt)[-i+j%N,1]
        end
    end
    plot(Plt,ylim = (-0.5, 1.5),title="Solución Recta orden 2",label = "t = $(j/N)")
end
gif(anim, "wave_deformation3.gif", fps = 25)

In [ ]:
Plt=copy(tsol6(0.002)[:,1])
TT=floor(Int, T)
anim = @animate for j in 1:25:N*TT
    for i in 1:N-1
        if (i-j%N)>=0
            Plt[N-i]=tsol6(j*dt)[N+j%N-i,1]
        else
            Plt[N-i]=tsol6(j*dt)[-i+j%N,1]
        end
    end
    plot(Plt,ylim = (-0.5, 1.5),title="Solución Recta orden 6",label = "t = $(j/N)")
end
gif(anim, "wave_deformation4.gif", fps = 25)

In [ ]:
anim = @animate for i in 0:100
    
    plot(x, sol2(i*T/100)[:,1], ylim = (-0.5, 1.5), label = "t = $(i*T/100)",title="Solución Lorenziana orden 2")
end

gif(anim, "wave_anim_1.gif", fps = 10)
    

In [ ]:
anim = @animate for i in 0:100
    
    plot(x, tsol2(i*T/100)[:,1], ylim = (-0.5, 1.5), label = "t = $(i*T/100)",title="Solución Recta orden 2")
end

gif(anim, "wave_anim_2.gif", fps = 10)

## Estimación de la energía

In [ ]:
TT=floor(Int, T)
Evec = copy(sol2(0.0)[:,1])
time = [i/N for i in 1:N*TT]
E2 = zeros(N*TT)
E4 = zeros(N*TT)
E6 = zeros(N*TT)
#integro numéricamente la solución en el espacio para cada tiempo
for i in 1:N*TT
    Evec .= sol2(i*dt)[:,1] .* sol2(i*dt)[:,1]
    E2[i] = sum(Evec) * dx

    Evec .= sol4(i*dt)[:,1] .* sol4(i*dt)[:,1]
    E4[i] = sum(Evec) * dx

    Evec .= sol6(i*dt)[:,1] .* sol6(i*dt)[:,1]
    E6[i] = sum(Evec) * dx
end
#Empezamos el gráfico en cero
E2 .= E2 .- E2[1]
E4 .= E4 .- E4[1]
E6 .= E6 .- E6[1]

Plt=plot()
plot!(Plt,time,E2,label="Orden 2")
plot!(Plt,time,E4,label="Orden 4")
plot!(Plt,time,E6,label="Orden 6")
plot!(Plt,title="Energía E(t) - E(0) Lorenziana")
#integro numéricamente la solución en el espacio para cada tiempo
for i in 1:N*TT
    Evec .= tsol2(i*dt)[:,1] .* tsol2(i*dt)[:,1]
    E2[i] = sum(Evec) * dx

    Evec .= tsol4(i*dt)[:,1] .* tsol4(i*dt)[:,1]
    E4[i] = sum(Evec) * dx

    Evec .= tsol6(i*dt)[:,1] .* tsol6(i*dt)[:,1]
    E6[i] = sum(Evec) * dx
end
#Empezamos el gráfico en cero
E2 .= E2 .- E2[1]
E4 .= E4 .- E4[1]
E6 .= E6 .- E6[1]

Plt2=plot()
plot!(Plt2,time,E2,label="Orden 2")
plot!(Plt2,time,E4,label="Orden 4")
plot!(Plt2,time,E6,label="Orden 6")
plot!(Plt2,title="Energía E(t) - E(0) Recta")
plot(Plt,Plt2,xlabel="t",ylabel="DeltaE(t)",legend = :bottomleft,size=(1200,600))

In [ ]:
#inicializo datos
N = 500            
L = 1.0            
dx = L/N           
T = 100.            
dt = 1. *dx        

r0=zeros(N,2)      
x = [dx*i for i in 0:N-1];    
#condición inicial
x0 = 0.4; x1 = 0.6
u = @view r0[:,1]
v = @view r0[:,2]

for i in 1:N
    x[i] = dx*(i-1)
    if x[i] > x0 && x[i] < x1
        u[i] = 1/(((x[i]-0.5)*75.0)^2+1.0)
        #lorenziana
        #u[i] = (x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250
        #polinomio
        #u[i] = (x[i]-x0)*5.0
        #recta
        #u[i] = exp(-(x[i]-0.5)^2/0.002)
        #gaussiana
        #u[i] = log(1.0/(1.0-(x[i]-x0)*5))-(x[i]-x0)*5*log((x[i]-x0)*5/(1.0-(x[i]-x0)*5))
        #logs
        v[i] = -u[i]
    end
end

D_2_per = create_D_2_per(N)
D_4_per = create_D_4_per(N)
D_6_per = create_D_6_per(N)
D_8_per = create_D_8_per(N)

p2 = (dx, D_2_per)
p4 = (dx, D_4_per)
p6 = (dx, D_6_per)
p8 = (dx, D_8_per)
prob2 = ODEProblem(F!,r0,(0.0,T),p2);
prob4 = ODEProblem(F!,r0,(0.0,T),p4);
prob6 = ODEProblem(F!,r0,(0.0,T),p6);
prob8 = ODEProblem(F!,r0,(0.0,T),p8);

#resuelvo
sol2 = solve(prob2,RK4(),dt=dt);
sol4 = solve(prob4,RK4(),dt=dt);
sol6 = solve(prob6,RK4(),dt=dt);
sol8 = solve(prob8,RK4(),dt=dt);
#plot
plot([r0[:,1],sol2(T)[:,1],sol4(T)[:,1],sol6(T)[:,1],sol8(T)[:,1]],label=["Exacta" "Segundo Orden" "Cuarto Orden" "Sexto Orden" "Octavo Orden"],title="Solución a tiempos finales T=100")

## Reducimos el número de puntos en la coordenada X

In [ ]:
N = 100            # Numeros de puntos en la discretización espacial
L = 1.0            # Intervalo espacial
dx = L/N           # dx
T = 10.            # Tiempo final
dt = 1. *dx        # Tomamos dt ≈ dx/speed_max, para mantener la condición CFL y
                   # Garantizar la estabilidad del algoritmo.

r0=zeros(N,2)      # Discretización de los campos u y v
x = [dx*i for i in 0:N-1];      # Coordenadas x, necesarias para determinar el dato inicial.

x0 = 0.4; x1 = 0.6
u = @view r0[:,1]
v = @view r0[:,2]
#el at view lo que hace es renombrar una columna para que pueda ser llamada por otro nombre, pero no crea ninguna variable nueva
for i in 1:N
    x[i] = dx*(i-1)
    if x[i] > x0 && x[i] < x1
        u[i] = 1/(((x[i]-0.5)*75.0)^2+1.0)
        #lorenziana
        #u[i] = (x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250
        #polinomio
        #u[i] = (x[i]-x0)*5.0
        #recta
        #u[i] = exp(-(x[i]-0.5)^2/0.002)
        #gaussiana
        #u[i] = log(1.0/(1.0-(x[i]-x0)*5))-(x[i]-x0)*5*log((x[i]-x0)*5/(1.0-(x[i]-x0)*5))
        #logs
        v[i] = -u[i]
    end
end
D_2_per = create_D_2_per(N)
D_4_per = create_D_4_per(N)
D_6_per = create_D_6_per(N)
D_8_per = create_D_8_per(N)

p2 = (dx, D_2_per)
p4 = (dx, D_4_per)
p6 = (dx, D_6_per)
p8 = (dx, D_8_per)
prob2 = ODEProblem(F!,r0,(0.0,T),p2);
prob4 = ODEProblem(F!,r0,(0.0,T),p4);
prob6 = ODEProblem(F!,r0,(0.0,T),p6);
prob8 = ODEProblem(F!,r0,(0.0,T),p8);

#resuelvo
sol2 = solve(prob2,RK4(),dt=dt);
sol4 = solve(prob4,RK4(),dt=dt);
sol6 = solve(prob6,RK4(),dt=dt);
sol8 = solve(prob8,RK4(),dt=dt);
plot([r0[:,1],sol2(T)[:,1],sol4(T)[:,1],sol6(T)[:,1],sol8(T)[:,1]],label=["Exacta" "Segundo Orden" "Cuarto Orden" "Sexto Orden" "Octavo Orden"],title="Soluciones (T=10 , N=100)")

**3)** Cambie el integrador temporal. ¿Qué sucede si usa Euler? Explique.

**4)** Grafique los errores con distintas resoluciones y diferencias finitas. 

## dt/dx >> 1  No converge el método

In [ ]:
N = 300            # Numeros de puntos en la discretización espacial
L = 1.0            # Intervalo espacial
dx = L/N           # dx
T = 10.            # Tiempo final
dt = 0.1        # Tomamos dt ≈ dx/speed_max, para mantener la condición CFL y
                   # Garantizar la estabilidad del algoritmo.

r0=zeros(N,2)      # Discretización de los campos u y v
x = [dx*i for i in 0:N-1];      # Coordenadas x, necesarias para determinar el dato inicial.

x0 = 0.4; x1 = 0.6
u = @view r0[:,1]
v = @view r0[:,2]
#el at view lo que hace es renombrar una columna para que pueda ser llamada por otro nombre, pero no crea ninguna variable nueva
for i in 1:N
    x[i] = dx*(i-1)
    if x[i] > x0 && x[i] < x1
        u[i] = 1/(((x[i]-0.5)*75.0)^2+1.0)
        #lorenziana
        #u[i] = (x[i] - x0)^4 * (x[i] - x1)^4 / (x1-x0)^8 * 250
        #polinomio
        #u[i] = (x[i]-x0)*5.0
        #recta
        #u[i] = exp(-(x[i]-0.5)^2/0.002)
        #gaussiana
        #u[i] = log(1.0/(1.0-(x[i]-x0)*5))-(x[i]-x0)*5*log((x[i]-x0)*5/(1.0-(x[i]-x0)*5))
        #logs
        v[i] = -u[i]
    end
end
D_2_per = create_D_2_per(N)
D_4_per = create_D_4_per(N)
D_6_per = create_D_6_per(N)
D_8_per = create_D_8_per(N)

p2 = (dx, D_2_per)
p4 = (dx, D_4_per)
p6 = (dx, D_6_per)
p8 = (dx, D_8_per)
prob2 = ODEProblem(F!,r0,(0.0,T),p2);
prob4 = ODEProblem(F!,r0,(0.0,T),p4);
prob6 = ODEProblem(F!,r0,(0.0,T),p6);
prob8 = ODEProblem(F!,r0,(0.0,T),p8);

#resuelvo
sol2 = solve(prob2,RK4(),dt=dt, adaptive=false);
sol4 = solve(prob4,RK4(),dt=dt, adaptive=false);
sol6 = solve(prob6,RK4(),dt=dt, adaptive=false);
sol8 = solve(prob8,RK4(),dt=dt, adaptive=false);
plot([r0[:,1],sol2(T)[:,1],sol4(T)[:,1],sol6(T)[:,1],sol8(T)[:,1]],label=["Exacta" "Segundo Orden" "Cuarto Orden" "Sexto Orden" "Octavo Orden"],title="Soluciones (T=10 , N=100)")

**5)** Haciendo diferencias de aproximaciones con distintas resoluciones y tomando normas apropiadas, vea cual es el orden de convergencia del método.

In [ ]:
function getsolution4(N,T,order)
    L = 1.0
    dx = L/N
    dt = 1e-4
    r0=zeros(N,2)
    x = [dx*i for i in 0:N-1];
    x0 = 0.4; x1 = 0.6
    u = @view r0[:,1]
    v = @view r0[:,2]
    for i in 1:N
        x[i] = dx*(i-1)
        if x[i] > x0 && x[i] < x1
            u[i] = (x[i] - x0)^8 * (x[i] - x1)^8 / (x1-x0)^16 * 250
        
            v[i] = -u[i]
        end
    end
    if order==2
        D_2_per_ = create_D_2_per(N)

        p2_ = (dx, D_2_per_)
        solving = ODEProblem(F!,r0,(0.0,T),p2_);
        solution = solve(solving,RK4(),dt=dt, saveat=0.001, adaptive=false);
    end
    if order==4
        D_4_per_ = create_D_4_per(N)

        p4_ = (dx, D_4_per_)
        solving = ODEProblem(F!,r0,(0.0,T),p4_);
        solution = solve(solving,RK4(),dt=dt, saveat=0.001, adaptive=false);
    end
    if order==6
        D_6_per_ = create_D_6_per(N)

        p6_ = (dx, D_6_per_)
        solving = ODEProblem(F!,r0,(0.0,T),p6_);
        solution = solve(solving,RK4(),dt=dt, saveat=0.001, adaptive=false);
    end
    if order==8
        D_8_per_ = create_D_8_per(N)

        p8_ = (dx, D_8_per_)
        solving = ODEProblem(F!,r0,(0.0,T),p8_);
        solution = solve(solving,RK4(),dt=dt, saveat=0.001, adaptive=false);
    end
    return r0 , solution
end
function detQ(t,sol_un,sol_do,sol_tre)
    soldt = copy(sol_un(t)[:,1])
    sol2dt = copy(sol_do(t)[1:2:end,1])
    sol4dt = copy(sol_tre(t)[1:4:end,1])
    sol = sum(abs.(soldt-sol2dt)) / sum(abs.(sol2dt-sol4dt))
    Q=sol
    return Q
end

In [ ]:
N=200
T=5.
r0 , sol_un = getsolution4(N,T,2)
r02,sol_do = getsolution4(2*N,T,2)
r04,sol_tre = getsolution4(4*N,T,2)

plot([r0[:,1],sol_un(T)[:,1],sol_do(T)[1:2:end,1],sol_tre(T)[1:4:end,1]],label=["Exacta" "dt" "dt/2" "dt/4"],title="Soluciones Orden 2")



In [ ]:
Qvec=zeros(50)
time=zeros(50)
for i in 1:50
    Q=detQ(i*0.1,sol_un,sol_do,sol_tre)
    time[i]=i*0.1
    Qvec[i]=Q
end
plot(time,Qvec,label="Q(t)",title="Q orden 2")

In [ ]:
N=200
T=5.
r0 , sol_un = getsolution4(N,T,6)
r02,sol_do = getsolution4(2*N,T,6)
r04,sol_tre = getsolution4(4*N,T,6)

plot([r0[:,1],sol_un(T)[:,1],sol_do(T)[1:2:end,1],sol_tre(T)[1:4:end,1]],label=["Exacta" "dt" "dt/2" "dt/4"],title="Soluciones Orden 6")


In [ ]:
Qvec=zeros(50)
time=zeros(50)
for i in 1:50
    Q=detQ(i*0.1,sol_un,sol_do,sol_tre)
    time[i]=i*0.1
    Qvec[i]=Q
end
plot(time,Qvec,label="Q(t)",title="Q orden 6")

In [ ]:
N=200
T=5.
r0 , sol_un = getsolution4(N,T,8)
r02,sol_do = getsolution4(2*N,T,8)
r04,sol_tre = getsolution4(4*N,T,8)

plot([r0[:,1],sol_un(T)[:,1],sol_do(T)[1:2:end,1],sol_tre(T)[1:4:end,1]],label=["Exacta" "dt" "dt/2" "dt/4"],title="Soluciones Orden 8")


In [ ]:
Qvec=zeros(50)
time=zeros(50)
for i in 1:50
    Q=detQ(i*0.1,sol_un,sol_do,sol_tre)
    time[i]=i*0.1
    Qvec[i]=Q
end
plot(time,Qvec,label="Q(t)",title="Q orden 8")

In [ ]:
N=200
T=5.
r0 , sol_un = getsolution4(N,T,4)
r02,sol_do = getsolution4(2*N,T,4)
r04,sol_tre = getsolution4(4*N,T,4)

plot([r0[:,1],sol_un(T)[:,1],sol_do(T)[1:2:end,1],sol_tre(T)[1:4:end,1]],label=["Exacta" "dt" "dt/2" "dt/4"],title="Soluciones Orden 4")


In [ ]:
Qvec=zeros(50)
time=zeros(50)
for i in 1:50
    Q=detQ(i*0.1,sol_un,sol_do,sol_tre)
    time[i]=i*0.1
    Qvec[i]=Q
end
plot(time,Qvec,label="Q(t)",title="Q orden 2")

**6)** Codifique la ecuación de burgers, $\frac{\partial u}{\partial t} = - u \frac{\partial u}{\partial x}$ y evolucione. Describa que sucede y explique.

In [ ]:
function Burg!(dr,r,p,t)
    # second order version
    dx, D = p
    h = 1. /dx
    u = @view r[:,1]
    v = @view r[:,2]
    du = @view dr[:,1]
    dv = @view dr[:,2]
    mul!(du, D, u, h, 0)
    du .= -u .* du
    #Nota: mul!(C, A, B, α, β) hace la operación α*A*B + β*C y la guarda en C
end

N = 500            # Numeros de puntos en la discretización espacial
L = 1.0            # Intervalo espacial
dx = L/N           # dx
T = 1.0           # Tiempo final
dt = 1. *dx        # Tomamos dt ≈ dx/speed_max, para mantener la condición CFL y
                   # Garantizar la estabilidad del algoritmo.

r0=zeros(N,2)      # Discretización de los campos u y v
x = [dx*i for i in 0:N-1];      # Coordenadas x, necesarias para determinar el dato inicial.


for i in 1:N
    x[i] = dx*(i-1)
    r0[i,1] = 0.3+sin(6*pi*x[i])/5.0
    
end

D_2_per = create_D_2_per(N)
D_4_per = create_D_4_per(N)
D_6_per = create_D_6_per(N)
D_8_per = create_D_8_per(N)

p2 = (dx, D_2_per)
p4 = (dx, D_4_per)
p6 = (dx, D_6_per)
p8 = (dx, D_8_per)
bur2 = ODEProblem(Burg!,r0,(0.0,T),p2);
bur4 = ODEProblem(Burg!,r0,(0.0,T),p4);
bur6 = ODEProblem(Burg!,r0,(0.0,T),p6);
bur8 = ODEProblem(Burg!,r0,(0.0,T),p8);

#resuelvo
burg2 = solve(bur2,RK4(),dt=dt);
burg4 = solve(bur4,RK4(),dt=dt);
burg6 = solve(bur6,RK4(),dt=dt);
burg8 = solve(bur8,RK4(),dt=dt);
tchoque=0.26
t2choque=0.29
befor=plot()
plot!(befor,[burg2(tchoque)[:,1],burg4(tchoque)[:,1],burg6(tchoque)[:,1],burg8(tchoque)[:,1]],label=["Segundo Orden" "Cuarto Orden" "Sexto Orden" "Octavo Orden"],title="Soluciones Burgers T=0.26")
after=plot()
plot!(after,[burg2(t2choque)[:,1],burg4(t2choque)[:,1],burg6(t2choque)[:,1],burg8(t2choque)[:,1]],label=["Segundo Orden" "Cuarto Orden" "Sexto Orden" "Octavo Orden"],title="Soluciones Burgers T=0.29")
plot(befor,after,xlabel="x",ylabel="u",size=(1200,600))

In [ ]:
anim = @animate for i in 0:200
    
    plot(x, burg2(i*T/200)[:,1],ylim=[0,1.5], label = "t = $(i*T/100)",title="Burgers orden 2")
end

gif(anim, "burgers_anim1.gif", fps = 20)

In [ ]:
anim = @animate for i in 0:200
    
    plot(x, burg6(i*T/200)[:,1],ylim=[0,1.5], label = "t = $(i*T/100)",title="Burgers orden 6")
end

gif(anim, "burgers_anim2.gif", fps = 20)

## Cosas Extra: Evolución de una señal ruidosa, tiene velocidad de grupo negativa

In [ ]:
#inicializo datos
N = 300            
L = 1.0            
dx = L/N           
T = 1.            
dt = 1. *dx        

r0=zeros(N,2)      
x = [dx*i for i in 0:N-1];    
#condición inicial
x0 = 0.4; x1 = 0.6
u = @view r0[:,1]
v = @view r0[:,2]

for i in 1:N
    x[i] = dx*(i-1)
    if x[i] > x0 && x[i] < x1
        u[i] = 1/(((x[i]-0.5)*75.0)^2+1.0)*(-1)^(i)
        
        v[i] = -u[i]
    end
end


D_4_per = create_D_4_per(N)

p4 = (dx, D_4_per)

prob4 = ODEProblem(F!,r0,(0.0,T),p4);

sol4 = solve(prob4,RK4(),dt=dt);

anim = @animate for i in 0:100
    
    plot(x, sol4(i*T/100)[:,1], ylim = (-0.8, 0.8), label = "t = $(i*T/100)",title="Solución ruidosa: se mueve al revés!")
end

gif(anim, "noisewave.gif", fps = 20)

In [ ]:
#inicializo datos
N = 300            
L = 1.0            
dx = L/N           
T = 1.            
dt = 1. *dx        

r0=zeros(N,2)      
x = [dx*i for i in 0:N-1];    
#condición inicial
x0 = 0.4; x1 = 0.6
u = @view r0[:,1]
v = @view r0[:,2]

for i in 1:N
    x[i] = dx*(i-1)
    if x[i] > x0 && x[i] < x1
        u[i] = 1/(((x[i]-0.5)*75.0)^2+1.0)*(1+(-1)^i)
        
        v[i] = -u[i]
    end
end


D_4_per = create_D_4_per(N)

p4 = (dx, D_4_per)

prob4 = ODEProblem(F!,r0,(0.0,T),p4);

sol4 = solve(prob4,RK4(),dt=dt);

anim = @animate for i in 0:100
    
    plot(x, sol4(i*T/100)[:,1], ylim = (-0.5, 1.25), label = "t = $(i*T/100)",title="Solución ruidosa")
end

gif(anim, "noisewave2.gif", fps = 20)

# la envolvente viaja en el otro sentido, y es un poco más rápida que la propagración del medio

In [ ]:
function Fuent!(dr,r,p,t)
    # second order version
    dx, D = p
    h = 1. /dx
    u = @view r[:,1]
    v = @view r[:,2]
    du = @view dr[:,1]
    dv = @view dr[:,2]
    mul!(du, D, v, h, 0)  #du/dt = h*Dv
    mul!(dv, D, u, h, 0)  #dv/dt = h*Du
    fuente=zeros(length(dv))
    x = [dx*i for i in 0:length(dv)-1];  
    for i in 1:length(dv)
        x[i] = dx*(i-1)
        if x[i]> 0.43 && x[i]<0.57 && t<1.2
            fuente[i] = 100.0
        end
    
    end
    dv.=dv.+fuente
    
    #Nota: mul!(C, A, B, α, β) hace la operación α*A*B + β*C y la guarda en C
end

N = 200            # Numeros de puntos en la discretización espacial
L = 1.0            # Intervalo espacial
dx = L/N           # dx
T = 3.0           # Tiempo final
dt = 1. *dx        # Tomamos dt ≈ dx/speed_max, para mantener la condición CFL y
                   # Garantizar la estabilidad del algoritmo.

r0=zeros(N,2)      # Discretización de los campos u y v
u = @view r0[:,1]
v = @view r0[:,2]
x = [dx*i for i in 0:N-1];      # Coordenadas x, necesarias para determinar el dato inicial.


for i in 1:N
    x[i] = dx*(i-1)
    u[i] = 0
    v[i]=-u[i]
    
end

D_2_per = create_D_2_per(N)
D_6_per = create_D_6_per(N)

p2 = (dx, D_2_per)
p6 = (dx, D_6_per)
fu2 = ODEProblem(Fuent!,r0,(0.0,T),p2);
fu6 = ODEProblem(Fuent!,r0,(0.0,T),p6);

#resuelvo
fue2 = solve(fu2,RK4(),dt=dt);
fue6 = solve(fu6,RK4(),dt=dt);

plot([fue2(T)[:,1],fue6(T)[:,1]],label=["Segundo Orden" "Sexto Orden"],title="Soluciones con fuente T=0.26")


In [ ]:
anim = @animate for i in 0:100
    
    plot(x, fue6(i*T/100)[:,1], ylim = (-7, 7), label = "t = $(i*T/100)",title="Solución con fuentes")
end

gif(anim, "surces_wave.gif", fps = 10)